Demonstration of quantEM config for gpus and such. 

More text to come eventually. 

Arthur McCray
Last remembered to update this line: July 9, 2025

In [1]:
from quantem.core import config

within the `config` module there is a `config` dictionary that has all the stuff. 
Access this dictionary with `get` and `set`
Can revert to defaults with `refresh`

In [2]:
print(config.config.keys())

dict_keys(['has_torch', 'has_cupy', 'device', 'precision', 'dtype_real', 'dtype_complex', 'verbose', 'cupy', 'mkl', 'warnings', 'viz'])


The most important and frequently used parts of config (especially within the quantEM code) are for checking if the environment has `cupy` and `torch`, in order to protect from import errors 

In [3]:
print("has cupy: ", config.get("has_cupy"))
print("has torch: ", config.get("has_torch"))
print("default device: ", config.get("device"))  # default device
config.set({"device": "gpu"})  # set device (gpu -> cuda:0)
print("set device: ", config.get("device"))  # current device
config.refresh()  # reset to defaults
print("device after refresh: ", config.get("device"))
print("config dict: \n", config.config)  # access to the raw dictionary

has cupy:  True
has torch:  True
default device:  cpu
set device:  cuda:0
device after refresh:  cpu
config dict: 
 {'has_torch': True, 'has_cupy': True, 'device': 'cpu', 'precision': 'float32', 'dtype_real': 'float32', 'dtype_complex': 'complex64', 'verbose': 1, 'cupy': {'fft-cache-size': '0 MB'}, 'mkl': {'threads': 2}, 'warnings': {'suppress-all-': False}, 'viz': {'interpolation': 'nearest', 'real_space_units': 'A', 'reciprocal_space_units': 'A^-1', 'cmap': 'gray', 'phase_cmap': 'magma', 'default_colors': '', 'colors': {'set': ['#3A7D44', '#E83F85', '#775AEB', '#ED8607', '#74D4B5', '#808080', '#C51D20', '#7C6A0A', '#00B4D8', '#774936'], 'paired': ['#3A7D44', '#A2C899', '#E83F85', '#FFA5C5', '#775AEB', '#C2B4F4', '#ED8607', '#F9C689', '#74D4B5', '#C2F0DE', '#808080', '#C0C0C0', '#C51D20', '#F28E8E', '#7C6A0A', '#C5B86A', '#00B4D8', '#80D9EB', '#774936', '#B38B7D']}}}


The cupy (and potentially future dask) settings aren't really tested very well, beyond cupy/gpus being available

In [4]:
print("cupy configs: ", config.get("cupy"))  # the cupy specific configs
print("cupy fft-cache-size: ", config.get("cupy.fft-cache-size"))  # sub-dicts can be gotten with .
config.set({"cupy.fft-cache-size": "10 MB"})
# there are equivalent methods of getting using the get method or accessing the config dict directly
print("set cupy fft-cache-size: ", config.get("cupy")["fft-cache-size"])
print("set cupy fft-cache-size: ", config.config["cupy"]["fft-cache-size"])

cupy configs:  {'fft-cache-size': '0 MB'}
cupy fft-cache-size:  0 MB
set cupy fft-cache-size:  10 MB
set cupy fft-cache-size:  10 MB


You can change the defaults (for this session/kernel) with `config.update_defaults`, this will not be overwritten by `refresh`

In [5]:
# set value and default that will be chosen by refresh until kernel is restarted
config.update_defaults({"device": 0})  # setting gpu with the index also works
print("Device: ", config.get("device"))
print("fft cache: ", config.get("cupy.fft-cache-size"))
config.refresh()
print("Device after refresh: ", config.get("device"))
print("fft cache after refresh: ", config.get("cupy.fft-cache-size"))

Device:  cuda:0
fft cache:  10 MB
Device after refresh:  cuda:0
fft cache after refresh:  0 MB


There are helpers for accessing the device, as that's the most frequently used part of the config. 

In [6]:
print(config.get("device"))
print(config.get_device())
print(config.device())
config.set_device(1)
print(config.device())

cuda:0
cuda:0
cuda:0
cuda:1


setting defaults or updating the config will throw an error if there isn't a gpu
Also the gpu device can be set by passing an integer or a pytorch style device or string  
Setting the gpu will set the device for `cupy` as well as the default device for `torch` (i.e. where a tensor goes if you send it `.to("cuda")`), but I prefer to be explicit and send tensors to the named device with `.to(config.get_device)`

In [7]:
try:
    config.update_defaults({"device": 1})
except ValueError as e:
    print(e)

try:
    config.set({"device": "cuda:5"})  ## throws an error if only 1 gpu
except ValueError as e:
    print(e)


Trying to set device cuda:5 but found 4 GPUs | has_cupy: True | has_torch: True


In [8]:
config.refresh()

In [9]:
config.update_defaults({"device": "cpu"}, config.config)
print(config.get("device"))
config.defaults  # the defaults are appended (chooses newest preferentially)

cpu


[{'has_torch': True, 'has_cupy': True},
 {'device': 'cpu',
  'precision': 'float32',
  'dtype_real': 'float32',
  'dtype_complex': 'complex64',
  'verbose': 1,
  'cupy': {'fft-cache-size': '0 MB'},
  'mkl': {'threads': 2},
  'warnings': {'suppress-all-': False},
  'viz': {'interpolation': 'nearest',
   'real_space_units': 'A',
   'reciprocal_space_units': 'A^-1',
   'cmap': 'gray',
   'phase_cmap': 'magma',
   'default_colors': '',
   'colors': {'set': ['#3A7D44',
     '#E83F85',
     '#775AEB',
     '#ED8607',
     '#74D4B5',
     '#808080',
     '#C51D20',
     '#7C6A0A',
     '#00B4D8',
     '#774936'],
    'paired': ['#3A7D44',
     '#A2C899',
     '#E83F85',
     '#FFA5C5',
     '#775AEB',
     '#C2B4F4',
     '#ED8607',
     '#F9C689',
     '#74D4B5',
     '#C2F0DE',
     '#808080',
     '#C0C0C0',
     '#C51D20',
     '#F28E8E',
     '#7C6A0A',
     '#C5B86A',
     '#00B4D8',
     '#80D9EB',
     '#774936',
     '#B38B7D']}}},
 {'device': 'cuda:0'},
 {'device': 'cuda:1'},
 {'dev

#### if you want to persist a config change, you can use the write method
All of the setting of defaults above is undone if you restart the kernel. When importing it looks for a user config file, and if one exists it will overwrite the quantem defaults with the user defaults. 

(The default location for the user file is "~/.config/quantem/config.yaml", in line with abtem which puts it at "~/.config/abtem/". The quantem defaults are stored in "quantem/core/quantem.yaml")

In [10]:
config.refresh()
print("device after refresh: ", config.get("device"))
config.set({"device": "cuda:0"})
config.write()
config.refresh()
print("device after refresh: ", config.get("device"))


device after refresh:  cpu
writing config to: /home/amccray/.config/quantem/config.yaml
device after refresh:  cuda:0


At this point if you restart the kernel, the default device will still be "cuda:0"

In [11]:
from pathlib import Path
from quantem.core import config

print(config.device())

cuda:0


For this demo we of course don't want to actually set the config, so lets undo that by removing the config file we just made

In [12]:
## reset to defaults by removing the config file
configfile = Path("~/.config/quantem/config.yaml").expanduser()
configfile.unlink()
config.refresh()
print("device after deleting the defaults file and refresh: ", config.get("device"))

device after deleting the defaults file and refresh:  cpu


## demo of working with multiple gpus and torch/cupy

In [13]:
import cupy as cp
import torch
from quantem.core import config

print("available gpus: ", torch.cuda.device_count())

available gpus:  4


In [14]:
config.refresh()
print("config device: ", config.get("device"))
arr0 = cp.arange(5)
print("arr0 is on the default cupy device, when config is cpu -> gpu0: ", arr0.device)
config.set({"device": 1})
print("setting config to gpu 1: ", config.get("device"))
print("this does not change the arr0 device, which is still: ", arr0.device)
arr1 = cp.arange(5)
print("arr1 is on the new device: ", arr1.device)

config device:  cpu
arr0 is on the default cupy device, when config is cpu -> gpu0:  <CUDA Device 0>
setting config to gpu 1:  cuda:1
this does not change the arr0 device, which is still:  <CUDA Device 0>
arr1 is on the new device:  <CUDA Device 1>


In [15]:
print(
    "working with cupy arrays on multiple devices will raise warnings (and probably give incorrect results)"
)
arrsum = arr0 + arr1
print("arrsum is on the new device: ", arrsum.device)

print("but the values aren't correct")
print("arr0: ", arr0)
print("arr1: ", arr1)
print("arrsum: ", arrsum)
print("note that we do get a performance warning from cupy, but it is not an error")

working with cupy arrays on multiple devices will raise warnings (and probably give incorrect results)
arrsum is on the new device:  <CUDA Device 1>
but the values aren't correct
arr0:  [0 1 2 3 4]
arr1:  [0 1 2 3 4]
arrsum:  [-1  0  1  2  3]
note that we do get a performance warning from cupy, but it is not an error


/tmp/ipykernel_2215707/1519630146.py:2: PerformanceWarning: The device where the array resides (0) is different from the current device (1). Peer access has been activated automatically.
  arrsum = arr0 + arr1


So basically, set the gpu device with `config.set({"device": XX})` before you start working with `cupy` arrays. 

With `torch` you have to explicitly set the tensor device either upon initialization or by moving the array, so changing the config is less of an issue. 

In [16]:
t1 = torch.tensor([1, 2, 3])
print("t1 device: ", t1.device)
t1 = t1.to(config.device())
print("t1 device: ", t1.device)
t1 = t1.cpu()
print("t1 device: ", t1.device)
t2 = torch.tensor([4, 5, 6], device=config.device())
print("t2 is built on the gpu: ", t2.device)
t3 = t1.cpu()
print(
    "setting the config device will set where the default 'cuda' sends the tensor, but it's better to be explicit"
)
t3 = t3.to("cuda")
print("t3 device: ", t3.device)

t1 device:  cpu
t1 device:  cuda:1
t1 device:  cpu
t2 is built on the gpu:  cuda:1
setting the config device will set where the default 'cuda' sends the tensor, but it's better to be explicit
t3 device:  cuda:1


-- end -- 